<a href="https://colab.research.google.com/github/joansantoso/aimlsurabaya/blob/main/KERAS_of_lora_tuning_Gemma2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
from google.colab import userdata
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### Install dependencies

In [4]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

### Select a backend

Keras is a high-level, multi-framework deep learning API designed for simplicity and ease of use. For this tutorial, configure the backend for JAX.


In [5]:
os.environ["KERAS_BACKEND"] = "jax"
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Import packages

Import Keras and KerasNLP.

In [6]:
import keras
import keras_nlp

## Dataset

In [7]:
train_contexts = [
    "Website iSTTS: https://www.istts.ac.id/",
    "Website pendaftaran iSTTS : https://pmb.istts.ac.id/manajemen/?menu=pendaftaran_online&page=pendaftaran_s1",
    "iSTTS merupakan sekolah teknik terbaik di Jawa Timur, dan sudah berdiri lebih dari 40 tahun untuk terus mengabdi pada bangsa dan negara Indonesia",
    "iSTTS memiliki dua fakultas yaitu fakultas teknik dan desain yang sangat selaras dengan teknologi yang sedang berkembang di dunia",
    "Untuk fakultas teknik terdapat D3 Sistem Informasi, S1 Teknik Elektro, S1 Informatika, S1 Teknik Industri, S1 Sistem Informasi Bisnis, S1 Informatika profesional (kelas malam)",
    "Untuk fakultas desain terdapat S1 Desain Produk, S1 Desain Komunikasi Visual",
    "iSTTS menyediakan Magister Teknologi Informasi",
    "iSTTS menyediakan Bachelor International Program yang merupakan program dual degree, pada program ini iSTTS bekerja sama dengan Swinburne university of technology di australia, Murdoch university di singapore, Dongseo university di korea",
    "D3 Sistem Informasi memfokuskan pada Web Desain, Jaringan Komputer, serta Basis Data dan Sistem Informasi untuk menghasilkan tenaga kerja profesional",
    "S1 Teknik Elektro memberikan konsep dan pengembangan perangkat keras maupun perangkat lunak yang terfokus pada Industrial Automation dan Industrial Internet of Things (IoT)",
    "S1 Informatika belajar memanfaatkan komputasi untuk problem solving menggunakan teknologi informasi yang memiliki 3 penjurusan yaitu, Software Technology, Computational Intelligence, dan Internet Technology",
    "S1 Teknik Industri memberikan pengajaran mengenai melakukan pengelolaan sistem yang memiliki penjurusan terkait Manufacturing System dan Quality Management",
    "S1 Sistem Informasi Bisnis yang menggabungkan Information Technology (IT) dan dunia Bisnis, serta memiliki 3 fokus pendidikan yaitu, Information Science, Multimedia and Game Technology, dan Social Media Business",
    "S1 Informatika profesional (kelas malam) menyediakan para perkerja yang melanjutkan studinya ataupun bagi para lulusan SMA yang ingin kuliah sambil bekerja dengan fokus pendidikan, antara lain Kewirausahaan serta Java dan .NET",
    "S1 Desain Produk memadukan desain dan teknologi otomasi untuk menghasilkan produk yang tidak hanya menarik secara tampilan, tetapi memiliki fungsi smart product. S1 Desain Produk memiliki 4 fokus pendidikan, yaitu Desain Produk Cerdas, Desain Kriya Modern, Desain Transportasi, dan Desain Interior",
    "S1 Desain Komunikasi Visual berfokus pada 3 pilar utama yaitu Seni, Teknologi, dan Bisnis. S1 Desain Komunikasi Visual memiliki 4 fokus pendidikan, yaitu Animasi, Perfilman, Ilustrasi, dan Fotografi",
    "Magister Teknologi Informasi memberikan pengetahuan dan wawasan tentang teknologi Informasi terbaru yang disertai dengan penggunaan dan pemanfaatan dalam dunia kerja. IT Utilization dan Artificial Intelligence merupakan 2 fokus pendidikan di Magister Teknologi Informasi",
    "iSTTS Memiliki 4 Laboratorium Komputer yang berada di L204, L304, L404, serta E401",
    "iSTTS Memiliki Research Laboratory untuk melakukan riset-riset mengenai teknologi-teknologi baru yang muncul",
    "iSTTS Juga memiliki fasilitas lain seperti Lapangan, Ruang Band, Theater, serta 2 Perpustakaan",
    "iSTTS Menyediakan sertifikasi yang dapat diambil oleh mahasiswa, seperti AWS, RedHat, MikroTik, Cisco, dan lain-lain",
    "Mahasiswa dapat masuk iSTTS tanpa melakukan tes masuk"
]

train_questions_answers = [
    {
        "context_index": 0,
        "question": "Apakah website iSTTS?",
        "answer": "https://www.istts.ac.id"
    },
    {
        "context_index": 0,
        "question": "Apakah iSTTS mempunyai website?",
        "answer": "Ya, iSTTS memiliki website yang terdapat di https://www.istts.ac.id"
    },
    {
        "context_index": 1,
        "question": "Apakah website pendaftaran iSTTS?",
        "answer": "https://pmb.istts.ac.id/manajemen/?menu=pendaftaran_online&page=pendaftaran_s1"
    },
    {
        "context_index": 1,
        "question": "Di manakah untuk melakukan pendaftaran iSTTS?",
        "answer": "Pendaftaran iSTTS bisa dilakukan di halaman website https://pmb.istts.ac.id/manajemen/?menu=pendaftaran_online&page=pendaftaran_s1"
    },
    {
        "context_index": 1,
        "question": "Apakah iSTTS memiliki website pendaftaran?",
        "answer": "Ya, iSTTS memiliki website pendaftaran yang terdapat di https://pmb.istts.ac.id/manajemen/?menu=pendaftaran_online&page=pendaftaran_s1"
    },
    {
        "context_index": 2,
        "question": "Apa itu iSTTS?",
        "answer": "iSTTS merupakan sekolah teknik terbaik di Jawa Timur dan sudah berdiri lebih dari 40 tahun untuk terus mengabdi pada bangsa dan negara Indonesia"
    },
    {
        "context_index": 2,
        "question": "Sudah berapa lama iSTTS berdiri?",
        "answer": "iSTTS sudah berdiri lebih dari 40 tahun"
    },
    {
        "context_index": 3,
        "question": "Apa saja fakultas yang ada di iSTTS?",
        "answer": "iSTTS memiliki dua fakultas yaitu fakultas teknik dan desain"
    },
    {
        "context_index": 4,
        "question": "Apa saja program studi yang ada di fakultas teknik iSTTS?",
        "answer": "Terdapat D3 Sistem Informasi, S1 Teknik Elektro, S1 Informatika, S1 Teknik Industri, S1 Sistem Informasi Bisnis, dan S1 Informatika Profesional (kelas malam)"
    },
    {
        "context_index": 5,
        "question": "Apa saja program studi yang ada di fakultas desain iSTTS?",
        "answer": "Terdapat S1 Desain Produk dan S1 Desain Komunikasi Visual"
    },
    {
        "context_index": 6,
        "question": "Apakah terdapat program studi untuk lanjutan di iSTTS?",
        "answer": "iSTTS juga menyediakan program Magister untuk Teknologi Informasi"
    },
    {
        "context_index": 7,
        "question": "Apakah iSTTS menyediakan program studi untuk ke luar negeri?",
        "answer": "Ya, iSTTS menyediakan program Bachelor International Program yang merupakan program dual degree"
    },
    {
        "context_index": 7,
        "question": "Apakah iSTTS memiliki hubungan kerja sama dengan universitas lain?",
        "answer": "Ya, iSTTS memiliki beberapa hubungan kerja sama dengan universitas di luar negeri untuk dual degree. Terdapat Swinburne university of technology di Australia, Murdoch university di Singapore, dan Dongseo university di Korea"
    },
    {
        "context_index": 7,
        "question": "Apa ada kerja sama dengan iSTTS di Australia?",
        "answer": "Ya ada, iSTTS memiliki kerja sama dengan Swinburne university of technology di Australia untuk program dual degree"
    },
    {
        "context_index": 7,
        "question": "Apa ada kerja sama dengan iSTTS di Singapore?",
        "answer": "Ya ada, iSTTS memiliki kerja sama dengan Murdoch univerrsity di Singapore untuk program dual degree"
    },
    {
        "context_index": 7,
        "question": "Apa ada kerja sama dengan iSTTS di Korea?",
        "answer": "Ya ada, iSTTS memiliki kerja sama dengan Dongseo university di Korea untuk program dual degree"
    },
    {
        "context_index": 8,
        "question": "Apabila ingin untuk fokus pada Web Desain program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi D3 Sistem Informasi berfokus pada Web Desain, Jaringan Komputer, Basis Data, dan Sistem Informasi"
    },
    {
        "context_index": 8,
        "question": "Apabila ingin untuk fokus pada Jaringan Komputer program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi D3 Sistem Informasi berfokus pada Web Desain, Jaringan Komputer, Basis Data, dan Sistem Informasi"
    },
    {
        "context_index": 8,
        "question": "Apabila ingin untuk fokus pada Basis Data program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi D3 Sistem Informasi berfokus pada Web Desain, Jaringan Komputer, Basis Data, dan Sistem Informasi"
    },
    {
        "context_index": 8,
        "question": "Apabila ingin untuk fokus pada Sistem Informasi program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi D3 Sistem Informasi berfokus pada Web Desain, Jaringan Komputer, Basis Data, dan Sistem Informasi"
    },
    {
        "context_index": 9,
        "question": "Apabila ingin untuk belajar mengenai perangkat keras program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Teknik Elektro memberikan konsep dan pengembangan perangkat keras maupun perangkat lunak"
    },
    {
        "context_index": 9,
        "question": "Apabila ingin untuk fokus pada Industrial Automation program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Teknik Elektro memiliki fokus penjurusan pada Industrial Automation"
    },
    {
        "context_index": 9,
        "question": "Apabila ingin untuk fokus pada IoT program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Teknik Elektro memiliki fokus penjurusan pada Internet of Things (IoT)"
    },
    {
        "context_index": 9,
        "question": "Apa saja penjurusan program studi Teknik Elektro di iSTTS?",
        "answer": "Di iSTTS program studi S1 Teknik Elektro memiliki 2 penjurusan, yaitu Industrial Automation dan Industrial Internet of Things (IoT)"
    },
    {
        "context_index": 10,
        "question": "Apabila ingin untuk belajar untuk coding program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Informatika belajar memanfaatkan komputasi untuk problem solving menggunakan teknologi informasi"
    },
    {
        "context_index": 10,
        "question": "Apabila ingin untuk fokus pada metode perancangan pembuatan aplikasi program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Informatika memiliki fokus pembelajaran pada Software Technology"
    },
    {
        "context_index": 10,
        "question": "Apabila ingin untuk fokus pada AI program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Informatika memiliki fokus pembelajaran pada Computational Intelligence"
    },
    {
        "context_index": 10,
        "question": "Apabila ingin untuk fokus pada jaringan internet program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Informatika memiliki fokus pembelajaran pada Internet Technology"
    },
    {
        "context_index": 10,
        "question": "Apa saja penjurusan program studi Informatika di iSTTS?",
        "answer": "Di iSTTS program studi S1 Informatika memiliki 3 penjurusan, yaitu Software Technology, Computational Intelligence, dan Internet Technology"
    },
    {
        "context_index": 11,
        "question": "Apa saja penjurusan program studi Teknik Industri di iSTTS?",
        "answer": "Di iSTTS program studi S1 Teknik Industri memiliki 2 penjurusan, yaitu Manufacturing System dan Quality Management"
    },
    {
        "context_index": 11,
        "question": "Apabila ingin untuk fokus pada pengelolaan sistem program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Teknik Industri memberikan pengajaran mengenai melakukan pengelolaan sistem"
    },
    {
        "context_index": 11,
        "question": "Apabila ingin untuk fokus pada mengelola Manufacturing System program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Teknik Industri memiliki penjurusan Manufacturing System"
    },
    {
        "context_index": 11,
        "question": "Apabila ingin untuk fokus pada Quality Management saat mengelola suatu sistem program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Teknik Industri memiliki penjurusan terkait Quality Management"
    },
    {
        "context_index": 12,
        "question": "Apa saja penjurusan program studi Sistem Informasi Bisnis di iSTTS?",
        "answer": "Di iSTTS program studi S1 Sistem Informasi Bisnis memiliki 3 penjurusan, yaitu Information Science, Multimedia and Game Technology, dan Social Media Business"
    },
    {
        "context_index": 12,
        "question": "Apabila ingin untuk belajar IT dan bisnis program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Sistem Informasi Bisnis yang menggabungkan Information Technology (IT) dan dunia Bisnis"
    },
    {
        "context_index": 12,
        "question": "Apabila ingin untuk belajar Game development program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Sistem Informasi Bisnis memiliki fokus pendidikan Multimedia and Game Technology yang bisa merambah ke Game Development"
    },
    {
        "context_index": 12,
        "question": "Apabila ingin untuk belajar mengenai AI tapi untuk bisnis program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Sistem Informasi Bisnis memiliki fokus pendidikan Information Science yang bertujuan untuk menghasilkan AI untuk dunia bisnis"
    },
    {
        "context_index": 12,
        "question": "Apabila ingin untuk belajar mengenai digital business program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Sistem Informasi Bisnis memiliki fokus pendidikan Social Media Business yang mengajarkan untuk membuat digital business"
    },
    {
        "context_index": 13,
        "question": "Apabila ingin untuk belajar namun sambil bekerja program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Informatika profesional (kelas malam) tersedia untuk orang yang ingin kuliah sambil bekerja"
    },
    {
        "context_index": 13,
        "question": "Apa saja fokus pendidikan S1 Informatika Profesional?",
        "answer": "Di iSTTS program studi S1 Informatika profesional (kelas malam) memiliki fokus pendidikan antara lain Kewirausahaan, Java, dan .NET"
    },
    {
        "context_index": 14,
        "question": "Apabila ingin belajar untuk cara yang baik menghasilkan produk program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Produk memadukan desain dan teknologi otomasi untuk menghasilkan produk yang tidak hanya menarik secara tampilan namun memiliki fungsi smart product"
    },
    {
        "context_index": 14,
        "question": "Apabila ingin belajar untuk merancang interior program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Produk memiliki penjurusan Desain Interior"
    },
    {
        "context_index": 14,
        "question": "Apabila ingin belajar untuk merancang produk cerdas program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Produk memiliki penjurusan Desain Produk Cerdas"
    },
    {
        "context_index": 14,
        "question": "Apa saja penjurusan pada program studi Teknik Industri di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Produk memiliki 4 penjurusan, yaitu Desain Produk Cerdas, Desain Kriya Modern, Desain Transportasi, dan Desain Interior"
    },
    {
        "context_index": 15,
        "question": "Apabila ingin belajar fotografi program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Komunikasi Visual memiliki penjurusan Fotografi"
    },
    {
        "context_index": 15,
        "question": "Apabila ingin belajar untuk membuat film program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Komunikasi Visual memiliki penjurusan Perfilman"
    },
    {
        "context_index": 15,
        "question": "Apabila ingin belajar untuk membuat animasi program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Komunikasi Visual memiliki penjurusan Animasi"
    },
    {
        "context_index": 15,
        "question": "Apabila ingin belajar melukis program studi apa yang harus saya ambil di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Komunikasi Visual memiliki penjurusan Ilustrasi"
    },
    {
        "context_index": 15,
        "question": "Apa saja penjurusan pada program studi Desain Komunikasi Visual di iSTTS?",
        "answer": "Di iSTTS program studi S1 Desain Komunikasi Visual memiliki penjurusan 4, yaitu Animasi, Perfilman, Ilustrasi, dan Fotografi"
    },
    {
        "context_index": 16,
        "question": "Apa saja fokus pembelajaran program studi Magister Teknologi Informasi di iSTTS?",
        "answer": "Di iSTTS program studi Magister Teknologi Informasi memiliki 2 fokus pembelajaran, yaitu IT Utilization dan Artificial Intelligence"
    },
    {
        "context_index": 17,
        "question": "Apa saja fasilitas di iSTTS?",
        "answer": "Di iSTTS terdapat 4 Laboratorium Komputer"
    },
    {
        "context_index": 18,
        "question": "Apakah saya bisa melakukan riset di iSTTS?",
        "answer": "Di iSTTS juga terdapat Research Laboratory untuk melakukan riset mengenai teknologi terbaru"
    },
    {
        "context_index": 19,
        "question": "Apa saja fasilitas di iSTTS?",
        "answer": "Di iSTTS terdapat Lapangan, Ruang Band, Theater, dan 2 Perpustakaan"
    },
    {
        "context_index": 20,
        "question": "Apakah iSTTS menyediakan program untuk sertifikasi?",
        "answer": "Di iSTTS menyediakan banyak program sertifikasi seperti AWS, RedHat, MikroTik, Cisco, dan lain-lain"
    },
    {
        "context_index": 21,
        "question": "Apakah saya perlu melakukan tes untuk masuk ke iSTTS?",
        "answer": "Mahasiswa dapat masuk iSTTS tanpa melakukan tes masuk"
    },
]

Preprocess the data. This tutorial uses a only use small amount of data. Consider using more training data for higher quality fine-tuning.

In [8]:
data = []
for features in train_questions_answers:
  features['context'] = train_contexts[features['context_index']]
  template = "<start_of_turn>user\n{question}<end_of_turn>\n<start_of_turn>model\nContext adalah {context}\nJadi jawabannya adalah {answer}<end_of_turn>\n"
  data.append(template.format(**features))

In [9]:
data[0]

'<start_of_turn>user\nApakah website iSTTS?<end_of_turn>\n<start_of_turn>model\nContext adalah Website iSTTS: https://www.istts.ac.id/\nJadi jawabannya adalah https://www.istts.ac.id<end_of_turn>\n'

## Load Model

In [10]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_instruct_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

##Inference before fine tuning


### Define formatting helper functions

In [11]:
from IPython.display import Markdown
import textwrap

def display_chat(prompt, text):
  formatted_prompt = "<font size='+1' color='brown'>🙋‍♂️<blockquote>" + prompt + "</blockquote></font>"
  text = text.replace('•', '  *')
  text = textwrap.indent(text, '> ', predicate=lambda _: True)
  formatted_text = "<font size='+1' color='teal'>🤖\n\n" + text + "\n</font>"
  return Markdown(formatted_prompt+formatted_text)

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


### Create a chat helper to manage the conversation state

In [12]:
class ChatState():
  """
  Manages the conversation history for a turn-based chatbot
  Follows the turn-based conversation guidelines for the Gemma family of models
  documented at https://ai.google.dev/gemma/docs/formatting
  """

  __START_TURN_USER__ = "<start_of_turn>user\n"
  __START_TURN_MODEL__ = "<start_of_turn>model\n"
  __END_TURN__ = "<end_of_turn>\n"

  def __init__(self, model, system=""):
    """
    Initializes the chat state.

    Args:
        model: The language model to use for generating responses.
        system: (Optional) System instructions or bot description.
    """
    self.model = model
    self.system = system
    self.history = []

  def add_to_history_as_user(self, message):
      """
      Adds a user message to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

  def add_to_history_as_model(self, message):
      """
      Adds a model response to the history with start/end turn markers.
      """
      self.history.append(self.__START_TURN_MODEL__ + message + self.__END_TURN__)

  def get_history(self):
      """
      Returns the entire chat history as a single string.
      """
      return "".join([*self.history])

  def get_full_prompt(self):
    """
    Builds the prompt for the language model, including history and system description.
    """
    prompt = self.get_history() + self.__START_TURN_MODEL__
    if len(self.system)>0:
      prompt = self.system + "\n" + prompt
    return prompt

  def send_message(self, message):
    """
    Handles sending a user message and getting a model response.

    Args:
        message: The user's message.

    Returns:
        The model's response.
    """
    self.add_to_history_as_user(message)
    prompt = self.get_full_prompt()
    response = self.model.generate(prompt, max_length=1024)
    result = response.replace(prompt, "")  # Extract only the new response
    self.add_to_history_as_model(result)
    return result

### Chatting

Query the model for checking it's knowledge about iSTTS

In [13]:
chat = ChatState(gemma_lm)
message = "Apakah kamu pernah tau mengenai iSTTS?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Apakah kamu pernah tau mengenai iSTTS?</blockquote></font><font size='+1' color='teal'>🤖

> Ya, saya tahu tentang iSTTS! 
> 
> **iSTTS** adalah singkatan dari **Indonesian Standard Test for Technical Skills**. 
> 
> Ini adalah tes standar yang digunakan untuk menilai keahlian teknis di Indonesia. 
> 
> **Berikut beberapa informasi tentang iSTTS:**
> 
> * **Tujuan:** Tes ini bertujuan untuk mengukur kemampuan teknis peserta dalam bidang tertentu, seperti:
>     * Teknologi informasi
>     * Teknik komputer
>     * Teknik manufaktur
>     * dll.
> * **Format:** Tes iSTTS biasanya berupa soal pilihan ganda, dengan beberapa jenis soal seperti:
>     * Soal berbasis teks
>     * Soal berbasis gambar
>     * Soal berbasis video
> * **Penggunaan:** Tes ini sering digunakan untuk:
>     * Seleksi calon karyawan di berbagai bidang
>     * Penilaian keahlian peserta pelatihan
>     * Penilaian keahlian peserta program studi
> * **Standar:** iSTTS memiliki standar yang ditetapkan oleh lembaga pemerintah Indonesia.
> 
> **Apakah kamu ingin tahu lebih lanjut tentang iSTTS?** 
> 
> Misalnya, kamu bisa bertanya tentang:
> 
> * **Cara persiapan untuk iSTTS**
> * **Contoh soal iSTTS**
> * **Informasi mengenai lembaga yang menerbitkan iSTTS**
> * **Informasi mengenai biaya iSTTS**
> 
> 
> Silahkan beri tahu saya apa yang ingin kamu ketahui! 
> <end_of_turn>
</font>

The model will not know the answer to a very specific topic




## LoRA Fine-tuning


In [14]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

In [15]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
)
gemma_lm.fit(data, epochs=20, batch_size=1)

Epoch 1/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 89s 745ms/step - loss: 0.7954 - sparse_categorical_accuracy: 0.4182
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 41s 343ms/step - loss: 0.6558 - sparse_categorical_accuracy: 0.4461
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 19s 348ms/step - loss: 0.5137 - sparse_categorical_accuracy: 0.5124
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 20s 353ms/step - loss: 0.4027 - sparse_categorical_accuracy: 0.5901
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 20s 357ms/step - loss: 0.3239 - sparse_categorical_accuracy: 0.6694
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 20s 360ms/step - loss: 0.2710 - sparse_categorical_accuracy: 0.7139
Epoch 7/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 20s 362ms/step - loss: 0.2364 - sparse_categorical_accuracy: 0.7401
Epoch 8/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 20s 366ms/step - loss: 0.2071 - sparse_categorical_accuracy: 0.7572
Epoch 9/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 20s 367ms/step - loss: 0.1819 - sparse_categorical_accuracy: 0.7796
Epoch 10/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 20s 366

## Inference after fine-tuning
After fine-tuning, responses follow the instruction provided in the prompt.

### Chatting

In [16]:
chat = ChatState(gemma_lm)
message = "Apakah kamu pernah tau mengenai apa itu iSTTS?"
display_chat(message, chat.send_message(message))

<font size='+1' color='brown'>🙋‍♂️<blockquote>Apakah kamu pernah tau mengenai apa itu iSTTS?</blockquote></font><font size='+1' color='teal'>🤖

> Context adalah iSTTS merupakan sekolah teknik terbaik di Jawa Timur, dan memiliki beberapa fasilitas terbaik seperti laboratorium terlengkap, serta memiliki beberapa program studi terbaik seperti program studi teknik mesin
> Jadi jawabannya adalah iSTTS merupakan sekolah teknik terbaik di Jawa Timur, dan memiliki beberapa fasilitas terbaik seperti laboratorium terlengkap, serta memiliki beberapa program studi terbaik seperti program studi teknik mesin<end_of_turn>

</font>